* [极客翻译官方文档中文版](http://wiki.jikexueyuan.com/project/tensorflow-zh/)
* GPU 运行仅 Linux, 且需安装 CUDA
* 暂时 Mac 运行

In [2]:
# 生成三维数据再用平面拟合的 example

import tensorflow as tf
import numpy as np

# 使用 NumPy 生成假数据(phony data), 总共 100 个点.
x_data = np.float32(np.random.rand(2, 100)) # 随机输入
y_data = np.dot([0.100, 0.200], x_data) + 0.300

# 构造一个线性模型
# 
b = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
y = tf.matmul(W, x_data) + b

# 最小化方差
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# 初始化变量
init = tf.initialize_all_variables()

# 启动图 (graph)
sess = tf.Session()
sess.run(init)

# 拟合平面
for step in xrange(0, 201):
    sess.run(train)
    if step % 20 == 0:
        print step, sess.run(W), sess.run(b)

# 得到最佳拟合结果 W: [[0.100  0.200]], b: [0.300]

0 [[ 0.59502947  0.29406738]] [-0.08221933]
20 [[ 0.24533205  0.28733519]] [ 0.17321162]
40 [[ 0.14800003  0.2368266 ]] [ 0.253957]
60 [[ 0.11685742  0.21391369]] [ 0.28331801]
80 [[ 0.10604358  0.20510143]] [ 0.29396018]
100 [[ 0.10218094  0.20185377]] [ 0.29781371]
120 [[ 0.10078867  0.20067181]] [ 0.29920864]
140 [[ 0.10028537  0.20024323]] [ 0.29971358]
160 [[ 0.10010327  0.20008805]] [ 0.29989633]
180 [[ 0.10003738  0.20003188]] [ 0.29996249]
200 [[ 0.10001352  0.20001155]] [ 0.29998642]


## 基本使用

* 使用图(graph)表示计算任务 (流程图)
    * 节点称为 `op` (operation) 
    * 使用 tensor 表示数据, 一个 tensor 是一个类型化的多维数组
    * 每个 op 获得 tensor 并计算, 产生 tensor
* 一个 tensorflow 图描述计算过程
    * 图必须在 session 里被启动, 在 context 中执行图
    * session 将 op 分发到 CPU/GPU 的设备上, 同时提供执行 op 方法
    * 将产生的 tensor 返回, python 中返回 `numpy.ndarray`
* 分**构建**和**执行**两个阶段
    * 如, 在构建阶段创建一个图表示和训练神经网络
    * 执行阶段反复执行途中的训练 op
* 通过变量(variable)维护状态 (?)
* feed 和 fetch 可以为任意操作赋值或获取数据

### 构建图

In [3]:
import tensorflow as tf

# 创建一个常量 op, 产生一个 1x2 矩阵. 这个 op 被作为一个节点加到默认图中.
#
# 构造器的返回值代表该常量 op 的返回值.
matrix1 = tf.constant([[3., 3.]])

# 创建另外一个常量 op, 产生一个 2x1 矩阵.
matrix2 = tf.constant([[2.],[2.]])

# 创建一个矩阵乘法 matmul op , 把 'matrix1' 和 'matrix2' 作为输入.
# 返回值 'product' 代表矩阵乘法的结果.
product = tf.matmul(matrix1, matrix2)

In [9]:
# 前面是构造了一个图
# 若要真正运算, 需要在 seesion 中启动这个图

### session 中启动图


In [10]:
# 启动默认图.
sess = tf.Session() # 一般默认图就够用

# 调用 sess 的 'run()' 方法来执行矩阵乘法 op, 传入 'product' 作为该方法的参数. 
# 上面提到, 'product' 代表了矩阵乘法 op 的输出, 传入它是向方法表明, 我们希望取回矩阵乘法 op 的输出.
#
# 整个执行过程是自动化的, session 负责传递 op 所需的全部输入. op 通常是并发执行的.
# 
# 函数调用 'run(product)' 触发了图中三个 op (两个常量 op 和一个矩阵乘法 op) 的执行.
#
# 返回值 'result' 是一个 `numpy.ndarray` 对象.
result = sess.run(product)
print result

# 任务完成, 关闭会话.
sess.close()

[[ 12.]]


In [11]:
# 除 close 外还可使用 with
with tf.Session() as sess:
        result = sess.run([product])
        print result

[array([[ 12.]], dtype=float32)]


In [ ]:
# 若需要使用 GPU, 需要明确指出
with tf.Session() as sess:
    with tf.device("/gpu:1"):
        matrix1 = tf.constant([[3.,3.]])
        matrix2 = tf.constant([[2.],[2.]])
        product = tf.matmul(matrix1, matrix2)
        ...

# "/cpu:0" 机器的 CPU
# "/gpu:0" 机器的第一个 GPU
# "/gpu:1" 机器的第2个 GPU
# ...

## 变量

图执行过程中的状态信息

In [12]:
# 使用变量实现一个简单的计数器example

# 创建一个变量, 初始化为标量 0.
state = tf.Variable(0, name="counter")

# 创建一个 op, 其作用是使 state 增加 1
one = tf.constant(1)
new_value = tf.add(state, one) # new values add +1
update = tf.assign(state, new_value) # update state by new_value

# 启动图后, 变量必须先经过`初始化` (init) op 初始化,
# 首先必须增加一个`初始化` op 到图中.
init_op = tf.initialize_all_variables() # 初始化op

# 启动图, 运行 op
with tf.Session() as sess:
    sess.run(init_op) # 运行 'init' op 初始化
    print sess.run(state) # 打印 'state' 的初始值
    for _ in range(3):
        sess.run(update) # 运行 op 更新 state
        print sess.run(state) 
# 在 run 之前, assign() 和 add() 都不会真正执行赋值操作

0
1
2
3


### Fetch

In [13]:
# 获取数据

input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.mul(input1, intermed)

with tf.Session() as sess:
    result = sess.run([mul, intermed])
    print result

[21.0, 7.0]


### Feed
* 临时替代图中任意操作中的 tensor, 对图中任何操作提交补丁
* 可作为 `run()`调用的参数
* 只在调用它的方法内有效, 方法结束, feed 消失

In [15]:
input1 = tf.placeholder(tf.float32) # 用 placeholder 标记
input2 = tf.placeholder(tf.float32)
output = tf.mul(input1, input2)

with tf.Session() as sess:
    print sess.run([output], feed_dict={input1:[7.], input2:[2.]})

[array([ 14.], dtype=float32)]
